In [1]:
import dash
from dash import html, dcc
from dash.dependencies import Input, Output
from dash.dash import no_update
import dash_bio as dashbio
import json

In [2]:
# Python functions for data wrangling

# read list of bands files available in the online database
from urllib.request import Request, urlopen, urlretrieve
from bs4 import BeautifulSoup

def read_url(url):
    bands = {}
    if not (url.endswith(".json") or url.endswith(".tsv")):    # when url is a directory
        url = url.replace(" ","%20")
        req = Request(url)
        a = urlopen(req).read()
        soup = BeautifulSoup(a, 'html.parser')
        x = (soup.find_all('a'))
        for i in x:
            file_name = i.extract().get_text()
            if file_name.endswith('.json') or file_name.endswith('.tsv'):
                bands[file_name.split('.')[0]] = file_name
        # read labels from the list.txt (if it exists)
        try:
            for line in urlopen(url+"/list.txt"):
                pair = line.decode('utf-8').strip().split(',')
                if pair[0] in bands.keys():
                    bands[pair[0]] = pair[1]
        except:
            pass
    else:                                                      # when url is a JSON file
        file_name = url.split("/")[-1]
        bands[file_name.split('.')[0]] = file_name
    return(bands)


# derive list of available chromosomes
import pandas as pd
import re

p = re.compile(r'(\d+)')

def extract_num(s, p, ret=0):
    search = p.search(s)
    if search:
        return int(search.groups()[0])
    else:
        return ret

def get_chromosomes(bands_file_url):
    chromosomes = ''
    try:
        data = json.loads(urlopen(bands_file_url).read())
        df = pd.DataFrame.from_dict(data['chrBands'])
        ch = set(df.iloc[:,0].str.split(' ', expand=True)[0].to_list())
        chromosomes = sorted(list(ch), key=lambda s: extract_num(s, p, float('inf')))
    except:
        print('Error: Extracting chromosomes from input data has failed. FILE: ', bands_file_url)
    return chromosomes


In [3]:
# Execute functions to provide initial settings

#default_url = "https://aedawid.github.io/ideogram/database/bands/"
default_url = "https://unpkg.com/ideogram/dist/data/bands/native/"
bands = read_url(default_url)
default_org = list(bands.keys())[0] + ".json"
input_path = default_url+default_org
chromosomes = get_chromosomes(input_path)
#default_anot = "https://aedawid.github.io/ideogram/database/annotations/"
default_anot = "https://unpkg.com/ideogram/dist/data/annotations/"
annotations = list(read_url(default_anot).values())
annotations.insert(0,'None')

In [4]:
# CSS styles

css_btn = {'font-size':'20px', 'background-color':'#008CBA', 'color':'white', 'border-radius':'8px', 'border':'1px solid #006B88', 'marginBottom':'10px'}
css_div = {'display':'inline-block'}
css_lab = {'color':'#008CBA', 'font-size':'16px', 'font-style':'italic', 'display':'inline-block'}
css_inp = {'marginBottom':'10px', 'width':'60%', 'display':'inline-block', 'font-size':'14px', 'padding':'6px 0'}
css_val = {'width':'30%', 'marginBottom':'20px'}
css_rad = {'padding': '1vh 2.5vw 0 0'}
css_col = {'display':'inline-block', 'width':'8%', 'padding':'3px 3px 3px 3px', 'font-size':'12px'}

# Application layout

app = dash.Dash(__name__)

app.layout = html.Div([
  html.Button('Show Options', id='options', n_clicks=0, style=css_btn),
  html.Div([
    html.Div([
        
        html.Label('Provide URL to online bands:', title="directory or file in JSON format", style={**css_lab, 'width':'65%'}),
        html.Label('Select bands data file:', title="list of available input files", style={**css_lab, 'width':'35%'}),
        dcc.Input(id="input-url", type="text", debounce=True,
            placeholder="e.g., https://unpkg.com/ideogram/dist/data/bands/native/",
            value=default_url, 
            style=css_inp),
      html.Div([
        dcc.Dropdown(id='dash-bands', value=list(bands.keys())[0], multi=False,
            options=[{'label': str(bands[i]), 'value': str(i)} for i in bands],
            style={'display':'block', 'width':'100%', 'verticalAlign':'top'}),
      ], style={'display':'inline-block', 'width':'36%', 'marginLeft':'3%'}),
        
      html.Label('Select chromosomes to display on the ideogram:', title="list of the names of chromosomes to display", style={**css_lab, 'width':'100%'}),
      dcc.Dropdown(
        id='dash-chromosomes',
        options=[{'label': str(i), 'value': str(i)} for i in chromosomes],
        multi=True,
        value=chromosomes
      ),
        
      html.Label('Provide URL to online annotations:', title="directory or file in JSON/TSV format",
                 style={**css_lab, 'width':'65%', 'marginTop':'10px'}),
      html.Label('Select annotations data file:', title="list of available annotation files",
                 style={**css_lab, 'width':'35%'}),
      dcc.Input(id="annot-url", type="text", debounce=True,
              placeholder="e.g., https://unpkg.com/ideogram/dist/data/annotations/",
              value=default_anot, 
              style={**css_inp, 'verticalAlign':'top'}),
      html.Div([
        dcc.Dropdown(id='dash-annots', value='None', multi=False,
          options=[{'label': str(i), 'value': str(i)} for i in annotations]),
      ], style={'display':'inline-block', 'width':'36%', 'marginLeft':'3%'}),
        
    ], id='data-opts', style={**css_div, 'width':'55%'}),
      
    html.Div([
      html.Label('Rotable:', title="allows rotation and zooming of the clicked chromosome", style={**css_lab, **css_val}),
      dcc.RadioItems(id='rotatable', options=['YES', 'NO'], value='YES', style={**css_div, 'width':'70%'}, labelStyle=css_rad),
      html.Label('Orientation:', title="select orientation of the diagram", style={**css_lab, **css_val}),
      dcc.RadioItems(id='orientation', options=['vertical', 'horizontal'], value='vertical', style={**css_div, 'width':'70%'}, labelStyle=css_rad),
      html.Label('Dimensions:', title="Max Height, Bar Width, Bar Gap", style={**css_lab, **css_val}),
      dcc.Input(id="chr-height", type="number", value=600, 
          style={'marginRight':'2%', 'width':'20%', 'display':'inline-block'}),
      dcc.Input(id="chr-width", type="number", value=20, 
          style={'marginRight':'2%', 'width':'20%', 'display':'inline-block'}),
      dcc.Input(id="chr-margin", type="number", value=10, 
        style={'marginBottom':'10px', 'width':'20%', 'display':'inline-block'}),
      html.Label('Genomic range:', title="range for a brush on a chromosome", style={**css_lab, **css_val}),
      dcc.Input(id="brush", type="text", placeholder="e.g., chr1:104325484-119977655",
          style={'marginRight':'2%', 'width':'66%', 'display':'inline-block'}),
      html.Label('Colorscale:', title="custom text for colors (10 fields)", style={**css_lab, **css_val, 'margin-bottom':'2px'}),
      dcc.Input(id="colorscale", type="text", debounce=True, placeholder="comma-separated string of 10 fields, e.g., a,b,c,d,e,f,g,h,i,j",
          style={'marginRight':'2%', 'width':'66%', 'display':'inline-block'}),
      html.Div(id='l1', style={**css_col,'background-color':'white', 'border':'1px solid black'}),
      html.Div(id='l2', style={**css_col,'background-color':'#bfbfbf'}),
      html.Div(id='l3', style={**css_col,'background-color':'#ababab'}),
      html.Div(id='l4', style={**css_col,'background-color':'#808080'}),
      html.Div(id='l5', style={**css_col,'background-color':'#575757', 'color':'white'}),
      html.Div(id='l6', style={**css_col,'background-color':'#404040', 'color':'white'}),
      html.Div(id='l7', style={**css_col,'background-color':'black', 'color':'white'}),
      html.Div(id='l8', style={**css_col,'background-color':'#ffdddd'}),
      html.Div(id='l9', style={**css_col,'background-color':'#c7c7ee'}),
      html.Div(id='l10', style={**css_col, 'width':'15%','background-color':'white'}),
    ], id='styling-opts', style={**css_div, 'width':'40%','marginLeft':'5%', 'verticalAlign':'top'}),
  ], id='optionsDiv'), 

  dashbio.Ideogram(id='dashbio-ideogram',),
])


In [ ]:
# Javascript clientside callbacks

app.clientside_callback(
    """
    function(largeValue1, largeValue2) {
        var x = document.getElementById("optionsDiv");
        var y = document.getElementById("options");
        if (x.style.display === "none") {
            x.style.display = "block";
            y.style.backgroundColor = "#D6F2FA";
            y.style.color = "#90B6C1";
            y.innerText = "Hide Options";
        } else {
            x.style.display = "none";
            y.style.backgroundColor = "#008CBA";
            y.style.color = "white";
            y.innerText = "Show Options";
        }
    }
    """,
    Output('optionsDiv', 'style'),
    Input('options', 'n_clicks'),
)


# Callbacks responsive to changes in Dash widgets (options panel)

@app.callback(
    [Output('dash-bands', 'options'), Output('dash-bands', 'value')],
    Input('input-url', 'value')
)
def update_bands_options(url):
    bands = read_url(url)
    return [bands, list(bands.keys())[0]]

@app.callback(
    [Output('dash-chromosomes', 'options'), Output('dash-chromosomes', 'value')],
    [Input('input-url', 'value'), Input('dash-bands', 'value')]
)
def update_chromosomes_options(url, band):
    if not band.endswith('.json'):
        band += ".json"
    if not url.endswith('.json'):
        url = str(url + '/' + band).replace("//", "/").replace(":/", "://")
    chromosomes = get_chromosomes(url)
    return [chromosomes, chromosomes]

@app.callback(
    [Output('dash-annots', 'options'), Output('dash-annots', 'value')],
    [Input('annot-url', 'value'), Input('dash-bands', 'value')]
)
def update_annotations_options(url, band):
    ctx = dash.callback_context
    trigger = ctx.triggered[0]['prop_id'].split('.')[0]
    if trigger == "annot-url":
        annotations = list(read_url(url).values())
        annotations.insert(0,'None')
        return [annotations, 'None']
    else:
        return [no_update, 'None']

@app.callback(
    [Output('l1', 'children'), Output('l2', 'children'),
     Output('l3', 'children'), Output('l4', 'children'),
     Output('l5', 'children'), Output('l6', 'children'),
     Output('l7', 'children'), Output('l8', 'children'),
     Output('l9', 'children'), Output('l10', 'children'),
    ],
    [Input('input-url', 'value'), Input('colorscale', 'value')]
)
def update_colorbar(url, desc):
    if desc != None:
        desc = desc.split(',')
        if len(desc) == 10:
            return([desc[0], desc[1], desc[2], desc[3], desc[4],
                   desc[5], desc[6], desc[7], desc[8], desc[9]])
    elif url.startswith("https://unpkg.com"):
        return([' gneg', ' gpos25', ' gpos33', ' gpos50', ' gpos66',
               ' gpos75', ' gpos100', ' acen', ' gvar', 'default CS'])
    else:
        return([' <0.14', ' <0.28', ' <0.42', ' <0.56', ' <0.70',
               ' <0.84', ' <1.00', ' <1.20', ' >1.21', 'of 2x mean'])
        
    
    
# Callbacks that directly change ideogram

@app.callback(
    Output('dashbio-ideogram', 'dataDir'),
    Input('input-url', 'value')
)
def update_dataDir(value):
    if value.endswith('.json') or value.endswith('.tsv'):
        value = str(value.rsplit('/', 1)[0] + "/").replace("//", "/").replace(":/", "://")
    return value

@app.callback(
    Output('dashbio-ideogram', 'organism'),
    Input('dash-bands', 'value')
)
def update_organism(value):
    return value.split('.')[0]

@app.callback(
    Output('dashbio-ideogram', 'chromosomes'),
    Input('dash-chromosomes', 'value')
)
def update_chromosomes(value):
    return value

@app.callback(
    Output('dashbio-ideogram', 'annotationsPath'),
    [Input('annot-url', 'value'), Input('dash-annots', 'value')]
)
def update_annotations(url, file):
    if file == 'None':
        return None
    elif url.endswith(".json") or url.endswith(".tsv"):
        return url
    else:
        return str(url + '/' + file).replace('//', '/').replace(":/", "://")

@app.callback(
    Output('dashbio-ideogram', 'brush'),
    Input('brush', 'value')
)
def update_genomic_range(value):
    return value

@app.callback(
    Output('dashbio-ideogram', 'rotatable'),
    Input('rotatable', 'value')
)
def update_rotatable(value):
    if value == 'YES':
        return True
    else:
        return False

@app.callback(
    Output('dashbio-ideogram', 'orientation'),
    Input('orientation', 'value')
)
def update_orientation(value):
    return value

@app.callback(
    [Output('dashbio-ideogram', 'chrHeight'), Output('dashbio-ideogram', 'chrWidth'), Output('dashbio-ideogram', 'chrMargin')],
    [Input('chr-height', 'value'), Input('chr-width', 'value'), Input('chr-margin', 'value')]
)
def update_chromosome_size(height, width, margin):
    return [height, width, margin]

app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [26/Oct/2022 15:56:43] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2022 15:56:43] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [26/Oct/2022 15:56:43] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2022 15:56:43] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2022 15:56:44] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2022 15:56:44] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2022 15:56:44] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2022 15:56:44] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2022 15:56:44] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2022 15:56:44] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2022 15:56:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2022 15:56:46] "PO

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/abadacz/Library/Miniforge3_x86/envs/graphing/lib/python3.9/site-packages/flask/app.py", line 2073, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/abadacz/Library/Miniforge3_x86/envs/graphing/lib/python3.9/site-packages/flask/app.py", line 1518, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/abadacz/Library/Miniforge3_x86/envs/graphing/lib/python3.9/site-packages/flask/app.py", line 1516, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/abadacz/Library/Miniforge3_x86/envs/graphing/lib/python3.9/site-packages/flask/app.py", line 1502, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "/Users/abadacz/Library/Miniforge3_x86/envs/graphing/lib/python3.9/site-packages/dash/dash.py", line 1344, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list)

127.0.0.1 - - [26/Oct/2022 15:56:51] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [26/Oct/2022 15:56:51] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2022 15:56:51] "POST /_dash-update-component HTTP/1.1" 200 -


Error: Extracting chromosomes from input data has failed. FILE:  /anopheles-gambiae.json
Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/abadacz/Library/Miniforge3_x86/envs/graphing/lib/python3.9/site-packages/flask/app.py", line 2073, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/abadacz/Library/Miniforge3_x86/envs/graphing/lib/python3.9/site-packages/flask/app.py", line 1518, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/abadacz/Library/Miniforge3_x86/envs/graphing/lib/python3.9/site-packages/flask/app.py", line 1516, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/abadacz/Library/Miniforge3_x86/envs/graphing/lib/python3.9/site-packages/flask/app.py", line 1502, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "/Users/abadacz/Library/Miniforge3_x86/envs/graphing/lib/python3.9/site-packages/dash/dash

127.0.0.1 - - [26/Oct/2022 15:56:58] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [26/Oct/2022 15:56:58] "POST /_dash-update-component HTTP/1.1" 200 -
